In [1]:
#Install all the Reqiuired Libraries and Packages 
import os
import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from python_speech_features import mfcc , logfbank
import librosa
import pickle
from scipy import signal
get_ipython().magic('matplotlib inline')

C:\Users\Pooja Sharma\AppData\Local\Temp\ipykernel_8052\2060519318.py:13: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('matplotlib inline')


In [2]:
#Import all the Required Packages and Libraies.
import soundfile
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [3]:
os.listdir(path=r'Dataset')
def getListOfFiles(dirName):
    listOfFile=os.listdir(dirName)
    allFiles=list()
    for entry in listOfFile:
        fullPath=os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles=allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

dirName = r'Dataset'
listOfFiles = getListOfFiles(dirName)
len(listOfFiles)

600

In [4]:
def envelope(y , rate, threshold):
    mask=[]
    y=pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10) ,  min_periods=1 , center = True).mean()
    for mean in y_mean:
        if mean>threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

In [5]:
#Next Step is In-Depth Visualisation of Audio Fiels and its certain features to plot for.
#They are the Plotting Functions to be called later. 
def plot_signals(signals):
    fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
    fig.suptitle('Time Series' , size=16)
    i=0
    for x in range(2):
        for y in range(5):
            axes[x,y].set_title(list(signals.keys())[i])
            axes[x,y].plot(list(signals.values())[i])
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1

def plot_fft(fft):
    fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
    fig.suptitle('Fourier Transform' , size=16)
    i=0
    for x in range(2):
        for y in range(5):
            data = list(fft.values())[i]
            Y,freq = data[0] , data[1]
            axes[x,y].set_title(list(fft.keys())[i])
            axes[x,y].plot(freq , Y)
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1
    
def plot_fbank(fbank):
    fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
    fig.suptitle('Filter Bank Coefficients' , size=16)
    i=0
    for x in range(2):
        for y in range(5):
            axes[x,y].set_title(list(fbank.keys())[i])
            axes[x,y].imshow(list(fbank.values())[i],cmap='hot', interpolation = 'nearest')
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1
            
def plot_mfccs(mfccs):
    fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
    fig.suptitle('Mel Frequency Capstrum  Coefficients' , size=16)
    i=0
    for x in range(2):
        for y in range(5):
            axes[x,y].set_title(list(mfccs.keys())[i])
            axes[x,y].imshow(list(mfccs.values())[i],
                             cmap='hot', interpolation = 'nearest')
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1

def calc_fft(y,rate):
    n = len(y)
    freq = np.fft.rfftfreq(n , d= 1/rate)
    Y= abs(np.fft.rfft(y)/n)
    return(Y,freq)

In [6]:
#Now Cleaning Step is Performed where:
#DOWN SAMPLING OF AUDIO FILES IS DONE  AND PUT MASK OVER IT AND DIRECT INTO CLEAN FOLDER
#MASK IS TO REMOVE UNNECESSARY EMPTY VOIVES AROUND THE MAIN AUDIO VOICE 
def envelope(y , rate, threshold):
    mask=[]
    y=pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10) ,  min_periods=1 , center = True).mean()
    for mean in y_mean:
        if mean>threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

In [7]:
#The clean Audio Files are redirected to Clean Audio Folder Directory 
import glob,pickle
for file in tqdm(glob.glob(r'Dataset\\**\\*.wav')):
    file_name = os.path.basename(file)
    signal , rate = librosa.load(file, sr=16000)
    mask = envelope(signal,rate, 0.0005)
    wavfile.write(filename= 'clean_speech\\'+str(file_name), rate=rate,data=signal[mask])

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [02:17<00:00,  4.38it/s]


In [8]:
#Feature Extraction of Audio Files Function 
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [9]:
animals = {
    'cat' : 'Cat',
    'dog' : 'Dog',
    'cow' : 'Cow',
    'donkey' : 'Donkey',
    'monkey' : 'Monkey',
    'sheep' : 'Sheep'}

observed_animals = ['Cat','Dog','Cow','Donkey','Monkey','Sheep']

In [14]:
#Load the data and extract features for each sound file
from glob import glob
import os
import glob
def load_data(test_size=0.33):
    x,y=[],[]
    answer = 0
    for file in glob.glob(r'C:\Users\Pooja Sharma\Documents\Animal Voice Classification\clean_speech/*.wav'):
        file_name=os.path.basename(file)
        animal=animals[file_name.split("-")[0]]
        feature=extract_feature(file, mfcc=True, chroma=True, mel=False)
        x.append(feature)
        y.append([animal,file_name])
          
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [15]:
#Split the dataset
import librosa
import numpy as np
x_train,x_test,y_trai,y_tes=load_data(test_size=0.33)
print(np.shape(x_train),np.shape(x_test), np.shape(y_trai),np.shape(y_tes))
y_test_map = np.array(y_tes).T
y_test = y_test_map[0]
test_filename = y_test_map[1]
y_train_map = np.array(y_trai).T
y_train = y_train_map[0]
train_filename = y_train_map[1]
print(np.shape(y_train),np.shape(y_test))
print(*test_filename,sep="\n")

(450, 53) (150, 53) (450, 2) (150, 2)
(450,) (150,)
sheep-38.wav
cow-86.wav
cat-63.wav
cow-46.wav
sheep-55.wav
cat-59.wav
monkey-76.wav
cat-54.wav
cat-65.wav
monkey-51.wav
dog-7.wav
monkey-28.wav
cow-3.wav
donkey-28.wav
sheep-86.wav
donkey-29.wav
dog-73.wav
cat-38.wav
sheep-91.wav
cow-9.wav
cat-45.wav
monkey-31.wav
sheep-93.wav
cat-44.wav
dog-61.wav
monkey-43.wav
sheep-99.wav
cat-79.wav
sheep-49.wav
sheep-46.wav
cow-77.wav
cat-68.wav
cat-42.wav
dog-19.wav
dog-9.wav
dog-18.wav
dog-89.wav
donkey-53.wav
cow-52.wav
cow-72.wav
monkey-30.wav
sheep-76.wav
cat-24.wav
cow-14.wav
monkey-94.wav
cow-5.wav
donkey-7.wav
cat-94.wav
donkey-1.wav
donkey-92.wav
cat-36.wav
cat-58.wav
donkey-84.wav
cow-92.wav
sheep-51.wav
sheep-21.wav
sheep-27.wav
dog-59.wav
sheep-16.wav
sheep-71.wav
dog-3.wav
dog-56.wav
cat-98.wav
cat-100.wav
cat-97.wav
monkey-84.wav
cat-74.wav
sheep-13.wav
cat-91.wav
cow-23.wav
donkey-36.wav
dog-75.wav
dog-99.wav
donkey-91.wav
donkey-62.wav
donkey-88.wav
cow-44.wav
dog-34.wav
cow-76.wav

In [16]:
#Get the shape of the training and testing datasets
#print((x_train, x_test))
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 53


In [111]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [112]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [113]:
#SAVING THE MODEL
import pickle
# Save the Modle to file in the current working directory
#For any new testing data other than the data in dataset

Pkl_Filename = "model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

In [114]:
# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    animal_voice_classification = pickle.load(file)

animal_voice_classification

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [115]:
x_test

array([[-2.74523682e+02,  5.27017136e+01, -1.07387856e+02, ...,
         7.24989057e-01,  6.08520269e-01,  0.00000000e+00],
       [-9.50216064e+01,  1.14300842e+02, -1.55699127e+02, ...,
         5.44866741e-01,  4.86567050e-01,  0.00000000e+00],
       [-1.34150192e+02,  1.09540070e+02,  2.73901176e+01, ...,
         6.30894959e-01,  6.87996984e-01,  0.00000000e+00],
       ...,
       [-2.95451752e+02,  1.51455185e+02, -4.77531166e+01, ...,
         1.50782406e-01,  4.12064731e-01,  0.00000000e+00],
       [-1.84664459e+02,  1.63794693e+02, -8.35135956e+01, ...,
         4.42214340e-01,  4.57932085e-01,  0.00000000e+00],
       [-1.82501755e+02,  1.07746399e+02, -7.91311417e+01, ...,
         7.68193781e-01,  8.11506808e-01,  0.00000000e+00]])

In [116]:
#predicting :
y_pred=animal_voice_classification.predict(x_test)
y_pred

array(['Sheep', 'Cow', 'Cat', 'Cow', 'Sheep', 'Cat', 'Monkey', 'Cat',
       'Dog', 'Monkey', 'Dog', 'Monkey', 'Cow', 'Donkey', 'Sheep',
       'Donkey', 'Dog', 'Cat', 'Sheep', 'Cow', 'Cat', 'Monkey', 'Sheep',
       'Cat', 'Dog', 'Monkey', 'Sheep', 'Cat', 'Sheep', 'Sheep', 'Cow',
       'Cat', 'Cat', 'Dog', 'Dog', 'Dog', 'Dog', 'Donkey', 'Cat', 'Cow',
       'Monkey', 'Sheep', 'Cat', 'Cow', 'Monkey', 'Cow', 'Donkey', 'Cat',
       'Donkey', 'Donkey', 'Cat', 'Cat', 'Donkey', 'Cow', 'Sheep',
       'Sheep', 'Sheep', 'Dog', 'Sheep', 'Sheep', 'Dog', 'Dog', 'Cat',
       'Cat', 'Cat', 'Monkey', 'Cat', 'Sheep', 'Cat', 'Cow', 'Donkey',
       'Dog', 'Dog', 'Donkey', 'Donkey', 'Donkey', 'Cow', 'Dog', 'Cow',
       'Sheep', 'Sheep', 'Cow', 'Monkey', 'Monkey', 'Cat', 'Cat', 'Sheep',
       'Donkey', 'Sheep', 'Sheep', 'Monkey', 'Cow', 'Cow', 'Cat', 'Cat',
       'Cow', 'Sheep', 'Monkey', 'Dog', 'Dog', 'Cat', 'Monkey', 'Sheep',
       'Cow', 'Monkey', 'Donkey', 'Monkey', 'Cat', 'Dog', 'Donkey', '

In [117]:
#Store the Prediction probabilities into CSV file 
import numpy as np
import pandas as pd
y_pred1 = pd.DataFrame(y_pred, columns=['predictions'])
y_pred1['file_names'] = test_filename
print(y_pred1)
y_pred1.to_csv('predictionfinal.csv')

    predictions    file_names
0         Sheep  sheep-38.wav
1           Cow    cow-86.wav
2           Cat    cat-63.wav
3           Cow    cow-46.wav
4         Sheep  sheep-55.wav
..          ...           ...
193         Dog    dog-70.wav
194         Cow    cow-29.wav
195         Dog    dog-62.wav
196         Cow    cow-93.wav
197       Sheep  sheep-80.wav

[198 rows x 2 columns]


In [118]:
model.score(x_test, y_test)

0.9696969696969697